In [2]:
import pandas as pd
import numpy as np 

In [3]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [4]:
train = pd.read_json("train.json")

In [5]:
print(train)

            cuisine     id                                        ingredients
0             greek  10259  [romaine lettuce, black olives, grape tomatoes...
1       southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
2          filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...
3            indian  22213                [water, vegetable oil, wheat, salt]
4            indian  13162  [black pepper, shallots, cornflour, cayenne pe...
5          jamaican   6602  [plain flour, sugar, butter, eggs, fresh ginge...
6           spanish  42779  [olive oil, salt, medium shrimp, pepper, garli...
7           italian   3735  [sugar, pistachio nuts, white almond bark, flo...
8           mexican  16903  [olive oil, purple onion, fresh pineapple, por...
9           italian  12734  [chopped tomatoes, fresh basil, garlic, extra-...
10          italian   5875  [pimentos, sweet pepper, dried oregano, olive ...
11          chinese  45887  [low sodium soy sauce, fresh ginger,

In [6]:
ingredients_set = set([])
for id, row in train.iterrows():
#     print(row['ingredients'])
    ingredients_set = ingredients_set | set(row['ingredients'])
    
print(ingredients_set)

{'ricotta', 'pecan meal', 'chinese radish', 'chicken flavor stuffing mix', 'strained yogurt', 'reduced sodium soy sauce', 'lemon soda', 'low sodium low fat pasta sauce', 'free-range eggs', 'cholesterol free egg substitute', 'beef rib roast', 'ground cloves', 'refrigerated piecrusts', 'mexican chocolate', 'low-fat balsamic vinaigrette', 'yardlong beans', 'rock salt', 'mixed fruit', 'mexican cooking sauce', 'shellfish', 'cake yeast', 'serrano chilies', 'french toast', 'flax seeds', 'asian pear', 'whitefish fillets', 'quick-cooking hominy grits', 'Estancia Pinot Noir', 'lime wedges', 'green chilies', 'green cardamom pods', 'stout', 'pippin apples', 'Jif Creamy Peanut Butter', 'Makers Mark Whisky', 'gooseberries', 'jamaican pumpkin', 'Kraft Slim Cut Mozzarella Cheese Slices', 'crabmeat', 'chicken flavored rice', 'red serrano peppers', 'crawfish', 'baby bok choy', 'egg whites', 'shredded zucchini', 'fresca', 'swiss chard', 'fronds', 'taro', 'cloud ear fungus', 'minced chicken', 'apricots', 

In [7]:
print(train.cuisine.value_counts())

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64


In [8]:
print(len(ingredients_set))

6714


In [9]:
ingredient_list = list(ingredients_set)

In [10]:
def preprocess(text):
    # replace one or more white-space characters with a space
    regex = re.compile(r"\s+")                               
    text = regex.sub(' ', text)    
    # lower case
    text = text.lower()          
    # remove digits and punctuation
    regex = re.compile(r"[%s%s]" % (string.punctuation, string.digits))
    text = regex.sub(' ', text)           
    # remove stop words
    sw = stopwords.words('english')
    #text = text.split()                                              
    text = ' '.join([w for w in text.split() if w not in sw]) 
    # remove short words
    ' '.join([w for w in text.split() if len(w) > 2])
    # lemmatize
    text = ' '.join([(WordNetLemmatizer()).lemmatize(w) for w in text.split()]) 
    return text

In [11]:
processed_list = []
for ingredient in ingredient_list:
    processed_list.append(preprocess(ingredient))

In [12]:
print(len(set(processed_list)))

6671


In [13]:
df = pd.DataFrame()
df['dish_id'] = train.id
print(df.head())

   dish_id
0    10259
1    25693
2    20130
3    22213
4    13162


In [14]:
for ingredient in processed_list:
    df[ingredient] = 0

In [15]:
df.head()

,dish_id,ricotta,pecan meal,chinese radish,chicken flavor stuffing mix,strained yogurt,reduced sodium soy sauce,lemon soda,low sodium low fat pasta sauce,free range egg,...,cheese soup,eye round steak,part skim mozzarella,granita,pure vanilla,mushroom broth,karo,lamb steak,corn flake cereal,orange flavored brandy
0,10259,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,25693,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20130,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,22213,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,13162,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
for id,row in train.iterrows():
    for word in row["ingredients"]:
        #df[df['campaign_id']==key][word]=1
        df.loc[df['dish_id'] ==row['id'], word] = 1
df

,dish_id,ricotta,pecan meal,chinese radish,chicken flavor stuffing mix,strained yogurt,reduced sodium soy sauce,lemon soda,low sodium low fat pasta sauce,free range egg,...,knorr pasta side cheesi cheddar,ranch-style seasoning,"rotini pasta, cook and drain",Daiya,Kraft Slim Cut Mozzarella Cheese Slices,Oscar Mayer Cotto Salami,Challenge Butter,LiptonÂ® Iced Tea Brew Family Size Tea Bags,Hidden ValleyÂ® Greek Yogurt Original RanchÂ® Dip Mix,crushed cheese crackers
0,10259,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,25693,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20130,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22213,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13162,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6602,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,42779,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3735,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16903,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,12734,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.sum().all()

False